In [22]:
import lovely_tensors as lt
from lovely_numpy import lovely, set_config
from rich import print

lt.monkey_patch()

set_config(repr=lovely)

In [23]:
from pathlib import Path

id = "asz41quw"
base_path = Path("/workspaces/repositories/jmp-peft/lltrainer")
print(base_path)

path = base_path / id / "predictions"
print(path)

/workspaces/repositories/jmp-peft/lltrainer

/workspaces/repositories/jmp-peft/lltrainer/asz41quw/predictions

In [24]:
pt_files = list(path.glob("*.pt"))
# print(pt_files)
# Sort by batch idx
pt_files.sort(key=lambda x: int(x.stem.rsplit("_", 1)[1][len("batch") :]))

In [35]:
import pandas as pd
from matbench_discovery import Key
from matbench_discovery.data import DATA_FILES

df_summary = pd.read_csv(DATA_FILES.wbm_summary).set_index(Key.mat_id)
df_summary

,formula,n_sites,volume,uncorrected_energy,e_form_per_atom_wbm,e_above_hull_wbm,bandgap_pbe,wyckoff_spglib_initial_structure,uncorrected_energy_from_cse,e_correction_per_atom_mp2020,e_correction_per_atom_mp_legacy,e_form_per_atom_uncorrected,e_form_per_atom_mp2020_corrected,e_above_hull_mp2020_corrected_ppd_mp,site_stats_fingerprint_init_final_norm_diff,wyckoff_spglib,unique_prototype
material_id,,,,,,,,,,,,,,,,,
wbm-1-1,Ac6 U2,8.0,317.48,-42.954,0.550,0.550,0.0,A3B_hP8_194_h_c:Ac-U,-42.955387,0.000,0.0,0.544327,0.544327,0.544310,0.128576,A3B_hP8_194_h_c:Ac-U,True
wbm-1-2,Ac1 Th3,4.0,137.42,-25.858,0.125,0.125,0.0,AB3_tI8_139_a_bd:Ac-Th,-25.858142,0.000,0.0,0.126190,0.126190,0.126152,0.051626,AB3_tI8_139_a_bd:Ac-Th,True
wbm-1-3,Ac1 U3,4.0,100.81,-34.934,0.768,0.768,0.0,AB3_tI8_139_a_bd:Ac-U,-34.934045,0.000,0.0,0.765339,0.765339,0.765340,0.061782,AB3_tI8_139_a_bd:Ac-U,True
wbm-1-4,Ag2 Au1 Hg1,4.0,76.26,-9.114,0.030,0.063,0.0,A2BC_cF16_225_c_a_b:Ag-Au-Hg,-9.114021,0.000,0.0,0.032195,0.032195,0.062869,0.067272,A2BC_cF16_225_c_a_b:Ag-Au-Hg,True
wbm-1-5,Ag2 Ba1 La1,4.0,124.15,-13.222,-0.178,0.100,0.0,A2BC_cF16_225_c_a_b:Ag-Ba-La,-13.222006,0.000,0.0,-0.175452,-0.175452,0.092193,0.050158,A2BC_cF16_225_c_a_b:Ag-Ba-La,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wbm-5-23304,I2 Te2 Zr2,6.0,203.35,-31.883,-0.911,0.021,0.0,ABC_oP6_59_a_a_b:I-Te-Zr,-31.885257,-0.267,0.0,-0.926076,-1.193076,0.019279,0.352661,ABC_oP6_59_a_a_b:I-Te-Zr,True
wbm-5-23305,Tl2 Zr2,4.0,92.16,-21.578,0.062,0.141,0.0,AB_cP2_221_a_b:Tl-Zr,-21.579593,0.000,0.0,0.060252,0.060252,0.139899,0.029811,AB_cP2_221_a_b:Tl-Zr,False
wbm-5-23306,Pb2 Tm1 Zr1,4.0,108.43,-21.085,-0.163,0.126,0.0,A2BC_tI8_139_d_a_b:Pb-Tm-Zr,-21.081939,0.000,0.0,-0.158310,-0.158310,0.113990,0.546470,A2BC_tI8_139_d_a_b:Pb-Tm-Zr,True


In [79]:
import torch

file = pt_files[100]

data = torch.load(file, map_location="cpu")
print(data.keys())

id = data["batch"]["id"]
df_summary.loc[id]

dict_keys(['relax_out', 'batch'])

,formula,n_sites,volume,uncorrected_energy,e_form_per_atom_wbm,e_above_hull_wbm,bandgap_pbe,wyckoff_spglib_initial_structure,uncorrected_energy_from_cse,e_correction_per_atom_mp2020,e_correction_per_atom_mp_legacy,e_form_per_atom_uncorrected,e_form_per_atom_mp2020_corrected,e_above_hull_mp2020_corrected_ppd_mp,site_stats_fingerprint_init_final_norm_diff,wyckoff_spglib,unique_prototype
material_id,,,,,,,,,,,,,,,,,
wbm-2-2624,Al1 Ce1 Co1 Fe1,4.0,61.32,-25.961,-0.177,0.213,0.0,ABCD_cF16_216_a_b_c_d:Al-Ce-Co-Fe,-25.960605,0.0,0.0,-0.175901,-0.175901,0.215537,0.0,ABCD_cF16_216_a_b_c_d:Al-Ce-Co-Fe,True


In [83]:
import gzip
import pickle

from pymatgen.analysis.phase_diagram import PatchedPhaseDiagram

with gzip.open(DATA_FILES.mp_patched_phase_diagram, "rb") as zip_file:
    ppd_mp: PatchedPhaseDiagram = pickle.load(zip_file)

ppd_mp

Cached 'mp_patched_phase_diagram' to '/root/.cache/matbench-discovery/1.0.0/mp/2023-02-07-ppd-mp.pkl.gz'
Loading 'mp_patched_phase_diagram' from cached file at '/root/.cache/matbench-discovery/1.0.0/mp/2023-02-07-ppd-mp.pkl.gz'


PatchedPhaseDiagram covering 46216 sub-spaces

In [90]:
df_cse = pd.read_json(DATA_FILES.wbm_computed_structure_entries)
df_cse = df_cse.set_index(Key.mat_id)
df_cse

,material_id,formula_from_cse,computed_structure_entry
0,wbm-1-1,Ac6 U2,{'@module': 'pymatgen.entries.computed_entries...
1,wbm-1-2,Ac1 Th3,{'@module': 'pymatgen.entries.computed_entries...
2,wbm-1-3,Ac1 U3,{'@module': 'pymatgen.entries.computed_entries...
3,wbm-1-4,Ag2 Au1 Hg1,{'@module': 'pymatgen.entries.computed_entries...
4,wbm-1-5,Ag2 Ba1 La1,{'@module': 'pymatgen.entries.computed_entries...
...,...,...,...
256958,wbm-5-23304,I2 Te2 Zr2,{'@module': 'pymatgen.entries.computed_entries...
256959,wbm-5-23305,Tl2 Zr2,{'@module': 'pymatgen.entries.computed_entries...
256960,wbm-5-23306,Pb2 Tm1 Zr1,{'@module': 'pymatgen.entries.computed_entries...
256961,wbm-5-23307,Pb2 Tm2 Zr2,{'@module': 'pymatgen.entries.computed_entries...


In [100]:
from pymatgen.entries.computed_entries import ComputedStructureEntry

cse_dict = df_cse.loc[id, Key.cse].item()
entry = ComputedStructureEntry.from_dict(cse_dict)
entry

wbm-2-2624 ComputedStructureEntry - Ce1 Al1 Fe1 Co1 (CeAlFeCo)
Energy (Uncorrected)     = -25.9606  eV (-6.4902  eV/atom)
Correction               = 0.0000    eV (0.0000   eV/atom)
Energy (Final)           = -25.9606  eV (-6.4902  eV/atom)
Energy Adjustments:
  None
Parameters:
  potcar_symbols         = ['PAW_PBE Co 06Sep2000', 'PAW_PBE Fe_pv 06Sep2000', 'PAW_PBE Ce 28Sep2000', 'PAW_PBE Al 04Jan2001']
  hubbards               = {}
  potcar_spec            = [{'titel': 'PAW_PBE Co 06Sep2000', 'hash': None}, {'titel': 'PAW_PBE Fe_pv 06Sep2000', 'hash': None}, {'titel': 'PAW_PBE Ce 28Sep2000', 'hash': None}, {'titel': 'PAW_PBE Al 04Jan2001', 'hash': None}]
  is_hubbard             = False
  run_type               = GGA
Data:

In [105]:
ppd_mp.get_e_above_hull(entry, allow_negative=True)

/opt/conda/envs/jmp-peft/lib/python3.11/site-packages/pymatgen/analysis/phase_diagram.py:1783: UserWarning: No suitable PhaseDiagrams found for Fe1 Co1 Ce1 Al1. Using SLSQP to find decomposition
  warnings.warn(f"{exc} Using SLSQP to find decomposition")


ValueError: Unable to get decomposition for wbm-2-2624 ComputedStructureEntry - Ce1 Al1 Fe1 Co1 (CeAlFeCo)
Energy (Uncorrected)     = -25.9606  eV (-6.4902  eV/atom)
Correction               = 0.0000    eV (0.0000   eV/atom)
Energy (Final)           = -25.9606  eV (-6.4902  eV/atom)
Energy Adjustments:
  None
Parameters:
  potcar_symbols         = ['PAW_PBE Co 06Sep2000', 'PAW_PBE Fe_pv 06Sep2000', 'PAW_PBE Ce 28Sep2000', 'PAW_PBE Al 04Jan2001']
  hubbards               = {}
  potcar_spec            = [{'titel': 'PAW_PBE Co 06Sep2000', 'hash': None}, {'titel': 'PAW_PBE Fe_pv 06Sep2000', 'hash': None}, {'titel': 'PAW_PBE Ce 28Sep2000', 'hash': None}, {'titel': 'PAW_PBE Al 04Jan2001', 'hash': None}]
  is_hubbard             = False
  run_type               = GGA
Data:

In [118]:
from typing import cast

from chgnet.graph import CrystalGraph
from chgnet.graph.converter import CrystalGraphConverter
from chgnet.model import CHGNet
from pymatgen.core import Structure

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CHGNet.load().to(device)
model.eval()

converter = CrystalGraphConverter(
    atom_graph_cutoff=12.0,
    bond_graph_cutoff=6.0,
).to(device)
converter.eval()


final_structure = cast(
    Structure, Structure.from_dict(data["relax_out"]["final_structure"])
)
graph = cast(CrystalGraph, converter(final_structure))

# Compute the energy
with torch.no_grad(), torch.inference_mode():
    out = model([graph.to(device)], task="e")
    print(out)
    energy = out["e"].view(-1).item()

# Hull energy
from typing import Any

from matbench_discovery.energy import get_e_form_per_atom
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility
from pymatgen.entries.computed_entries import ComputedStructureEntry

cse_dict: dict[str, Any] = df_cse.loc[data["batch"]["id"], Key.cse].item()
cse_dict["structure"] = final_structure.as_dict()
cse_dict["energy"] = energy
cse = ComputedStructureEntry.from_dict(cse_dict)
# print(cse)

processed = MaterialsProject2020Compatibility().process_entries(
    cse, verbose=True, clean=True
)
assert isinstance(processed, list) and len(processed) == 1
processed = processed[0]
# print(processed)

e_form_per_atom = get_e_form_per_atom(cse)
e_form_per_atom

# energy = data["relax_out"]["trajectory"]["energy"][-1]

# e_form = get_e_form_per_atom(
#     {
#         "energy": energy,
#         # "energy": data["relax_out"]["trajectory"]["energy"][-1],
#         "composition": final_structure.composition,
#     }
# )

print(energy, e_form_per_atom, data["batch"]["y_above_hull"])

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda:1


{'atoms_per_graph': tensor[1] i64 cuda:0 [4], 'e': tensor[1] cuda:0 [-6.409]}

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/envs/jmp-peft/lib/python3.11/site-packages/pymatgen/entries/compatibility.py:1027: UserWarning: Failed to guess oxidation states for Entry wbm-2-2624 (CeAlFeCo). Assigning anion correction to only the most electronegative atom.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 3418.34it/s]


-6.408584117889404 4.712103970527649 [0.215537]